## **RAG 기본 구조 이해하기**

### **1. 사전작업(Pre-processing) - 1~4 단계**

<img src='https://ifh.cc/g/VK9RJ9.png' width = "800">
<img src='https://velog.velcdn.com/images/jjlee6496/post/f4443dd4-cdca-4e4c-83e6-8ec515c21db5/image.png' width = "800">


사전 작업 단계에서는 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 하는 4단계를 진행

- 1단계 : 도큐먼트 로드 (Document Loader): 외부 데이터 소스에서 필요한 문서를 로드하고 초기 처리. 
    - 이것은 마치 책을 여러 권 챙겨 도서관에서 공부하는 것과 비슷. 학생이 공부하기 전에 필요한 책들을 책장에서 골라오는 과정.

- 2단계 : 텍스트 분할 (Text Splitter): 로드된 문서를 처리 가능한 작은 단위로 분할. 큰 책을 챕터별로 나누는 것과 유사.

- 3단계 : 임베딩 (Embedding): 각 문서 또는 문서의 일부를 벡터 형태로 변환하여, 문서의 의미를 수치화. 이는 책의 내용을 요약하여 핵심 키워드로 표현하는 것과 비슷.

- 4단계 : 벡터스토어(Vector Store) 저장: 임베딩된 벡터들을 데이터베이스에 저장. 이는 요약된 키워드를 색인화하여 나중에 빠르게 찾을 수 있도록 하는 과정.


### **2. RAG 수행(RunTime) - 5~8 단계**

<img src='https://ifh.cc/g/plorDw.png' width = "800">

<img src='https://velog.velcdn.com/images/jjlee6496/post/15b30fe5-1015-47b4-b837-d692f9a101fd/image.png' width = "800">


- 5단계 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 Retriever 정의.
    - Retriever는 검색 알고리즘
    - Dense: 유사도 기반 검색, Sparse: 키워드 기반 검색
- 6단계 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됨. 
    - 프롬프트 엔지니어링을 통하여 답변의 형식을 지정할 수 있음.
- 7단계 LLM: 모델을 정의.(GPT-3.5, GPT-4, Claude, etc..)
- 8단계 Chain: 프롬프트 - LLM - 출력 에 이르는 체인을 생성.

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


### **Load Documents**

In [10]:
loader = PyMuPDFLoader('data/test.pdf')
docs = loader.load()
print(f'{len(docs)} pages docs')

2 pages docs


### **Text Split**

In [22]:
# Split text into characters
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
split_docs = splitter.split_documents(docs)    # chunk_size : 몇 단어씩 자를 것인지, chunk_overlap : 몇 단어씩 겹칠 것인지

### **Embedding**

In [24]:
embeded = OpenAIEmbeddings()

### **VectorDB 생성**

In [ ]:
vector_store = FAISS.from_documents(documents=split_docs, embedding=embeded)

### **Retriever 생성**

In [41]:
retriver = vector_store.as_retriever()

### **프롬프트 생성(Create Prompt)**


In [44]:
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

### **언어모델 생성(Chain)**

In [45]:
llm = ChatGroq(model = 'gemma2-9b-it')

chain = (
    {'context' : retriver, 'question' : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### **출력**

In [46]:
response = chain.stream('사원들의 성명을 알려주세요')

for c in response:
    print(c, end='', flush=True)

문서에 나와있는 사원 명단은 다음과 같습니다:

* 김철수
* 이영희
* 박민준 


